In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
data_path  = '../input/vehicle-dataset-from-cardekho/car data.csv'
df = pd.read_csv(data_path)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop_duplicates(subset=None, keep='first', inplace=True)

In [ ]:
df.shape

In [ ]:
print(df['Fuel_Type'].unique())
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())

In [ ]:
import seaborn as sns
sns.heatmap(df.isnull(),yticklabels = False,cbar = False,cmap = 'viridis')

In [ ]:
df.columns

In [ ]:
final_data = df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_data.head()

In [ ]:
final_data['current_year'] = 2020

In [ ]:
final_data.head()

In [ ]:
final_data['Age_Car'] = final_data['current_year'] - final_data['Year']

In [ ]:
final_data.head()

In [ ]:
final_data.drop(['Year', 'current_year'], axis = 1, inplace = True)

In [ ]:
final_data.head()

In [ ]:
Dummies = pd.get_dummies(final_data[['Fuel_Type', 'Seller_Type', 'Transmission']], drop_first = True)
final_data = final_data.drop(['Fuel_Type', 'Seller_Type', 'Transmission'], axis = 1)
final_data = pd.concat([final_data, Dummies], axis = 1)

In [ ]:
final_data.head()

In [ ]:
final_data.corr()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
corrmat = final_data.corr()
plt.figure(figsize=(10,10))
#plot heat map
sns.heatmap(corrmat, annot=True, cmap="RdYlGn")

In [ ]:
sns.pairplot(final_data)

In [ ]:
final_data.columns

In [ ]:
plt.figure(figsize =(8,5))
sns.boxplot(x = final_data['Present_Price']);

In [ ]:
import numpy as np

In [ ]:
data = final_data['Present_Price']

In [ ]:
Present_Price_list = final_data['Present_Price'].tolist() 

In [ ]:
def detect_outlier(data):
    outlier = []
    threshold = 3
    mean = np.mean(data)
    std = np.std(data)
    for i in data:
        z_score = (i - mean)/std
        if np.abs(z_score)>threshold:
            outlier.append(i)
    return outlier

In [ ]:
outlier_pt = detect_outlier(Present_Price_list)

In [ ]:
outlier_pt

In [ ]:
Kms_Driven_list = final_data['Kms_Driven'].tolist() 

In [ ]:
Kms_Driven_list_outlier = detect_outlier(Kms_Driven_list)

In [ ]:
Kms_Driven_list_outlier

In [ ]:
final_data.drop(final_data[final_data['Present_Price'] > 35.96].index, inplace = True) 

In [ ]:
final_data.shape

In [ ]:
final_data.drop(final_data[final_data['Kms_Driven'] > 197175].index, inplace = True) 

In [ ]:
final_data.shape

In [ ]:
final_data.head()

In [ ]:
X = final_data.drop(['Selling_Price'], axis = 1)
y = final_data['Selling_Price']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
### Feature Importance

from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 50, num = 10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model_rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = model_rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions=rf_random.predict(X_test)

In [ ]:

sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
metrics.r2_score(y_test, predictions)